In [ ]:
# ======================================
# STEP 1: Import Libraries
# ======================================
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.metrics import mean_squared_error, r2_score, roc_curve, auc

# ======================================
# STEP 2: Load Dataset
# ======================================
df = pd.read_csv('/content/drive/MyDrive/dataset.csv')

X = df.iloc[:, :-1].values
y = df.iloc[:, -1].values

# ======================================
# STEP 3: Feature Scaling (Manual)
# ======================================
X = (X - X.mean(axis=0)) / X.std(axis=0)

# ======================================
# STEP 4: Train-Test Split
# ======================================
split = int(0.8 * len(X))
X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]

# ======================================
# STEP 5: Elastic Net From Scratch
# ======================================
class ElasticNetScratch:
    def __init__(self, lr=0.01, alpha=0.1, l1_ratio=0.5, epochs=1000):
        self.lr = lr
        self.alpha = alpha
        self.l1_ratio = l1_ratio
        self.epochs = epochs

    def fit(self, X, y):
        self.w = np.zeros(X.shape[1])
        self.b = 0

        for _ in range(self.epochs):
            y_pred = X @ self.w + self.b
            error = y_pred - y

            l1 = self.l1_ratio * np.sign(self.w)
            l2 = (1 - self.l1_ratio) * self.w

            dw = (X.T @ error) / len(y) + self.alpha * (l1 + l2)
            db = np.mean(error)

            self.w -= self.lr * dw
            self.b -= self.lr * db

    def predict(self, X):
        return X @ self.w + self.b

# ======================================
# STEP 6: Train Model
# ======================================
model = ElasticNetScratch()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

# ======================================
# STEP 7: Evaluation
# ======================================
print("MSE:", mean_squared_error(y_test, y_pred))
print("R² :", r2_score(y_test, y_pred))

# ======================================
# STEP 8: Visualization
# ======================================
sns.scatterplot(x=y_test, y=y_pred)
plt.title("Elastic Net Scratch: Actual vs Predicted")
plt.show()

# ======================================
# STEP 9: ROC Curve (Thresholded)
# ======================================
threshold = np.median(y_test)
y_bin = (y_test >= threshold).astype(int)

fpr, tpr, _ = roc_curve(y_bin, y_pred)
plt.plot(fpr, tpr, label=f"AUC={auc(fpr,tpr):.2f}")
plt.plot([0,1],[0,1],'--')
plt.legend()
plt.show()
